### Input File extraction and XSLT transformation with lxml

In [90]:
from pathlib import Path
from lxml import etree as et

file_read = Path.cwd() / "files" / "sharedparse" / "input" / "GetAccountSchema.sharedparse"
with open(file_read, "r") as reader:
    data = et.parse(reader)

# Pass the data through XSLT to remove the namespaces
xslt_doc = Path.cwd() / "files" / "common" / "xslt_doc.xml"
with open(xslt_doc, "r") as reader:
    xslt = et.XSLT(et.parse(reader))

transformed_data = xslt(data)

transformed_doc = Path.cwd() / "files" / "sharedparse" / "generated" / "transformed_doc.xml"
transformed_data.write(transformed_doc)

### Method to dump JSON contents into a file

In [91]:
def write_json(content, path):
    with open(path, "w", encoding="utf-8") as json_file:
        json.dump(content, json_file, ensure_ascii=False, indent=4)

### Build the JSON dictionary

In [92]:
import json

with open(transformed_doc, "r") as reader:
    root = et.parse(reader).getroot()

tree = {}
lines = ""
if root.findtext("resourceType") == "ae.shared.ParseSharedResource":
    config = root.find("config")

    tree['name'] = root.findtext("name")
    tree['length'] = config.findtext("LineLength")
    tree['values'] = {}

    values = config.findtext("OffsetString")
    lines = values.strip().splitlines()

    size = 6
    chunks = {}
    for i in range(0, len(lines), size):
        chunk = lines[i:i + size]
        chunks[i//size + 1] = chunk

    tree['values'] = chunks

file_path = Path.cwd() / "files" / "sharedparse" / "generated" / "tree.json"
write_json(tree, file_path)